In [1]:
import pandas as pd
import requests
import json
import re
import sys
import numpy as np
import os
import nltk
import pickle
from sklearn.utils import shuffle
from importlib import reload
from langdetect import detect
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from dateutil.parser import parse
import spacy
from time import time

sys.path.append('../src')

from commons import elastic
from text_processing import text_normalizer
text_normalizer = reload(text_normalizer)
from text_processing import duplicate_finder
from utilities import excel_writer, excel_reader, utils
from text_processing import abbreviations_resolver
abbreviations_resolver = reload(abbreviations_resolver)

stemmer = SnowballStemmer("english")
lmtzr = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')

_abbreviations_resolver = abbreviations_resolver.AbbreviationsResolver([])
_abbreviations_resolver.load_model("../model/abbreviations_dicts")

C:\Users\Maryia_Ivanina\Anaconda3\envs\cornell\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Prepare data from txt to excel files

In [21]:
all_docs = {}
global_id = 0
mappings_for_columns = {
    "T1  - ": "title",
    "A1  - ": "authors",
    "AB  - ": "abstract",
    "DO  - ": "doi",
    "KW  - ": "keywords",
    "PY  - ": "year",
    "UR  - ": "url"
}
for file in os.listdir("../tmp/folder_with_egm_files/"):
    filename = file.split(".")[0]
    file = os.path.join("../tmp/folder_with_egm_files/", file)
    keywords_started = False
    with open(file, "r", encoding="utf-8") as f:
        for line in f.readlines():
            if not line.strip():
                global_id += 1
                continue
            if global_id not in all_docs:
                all_docs[global_id] = {}
                for key_ in mappings_for_columns:
                    all_docs[global_id][mappings_for_columns[key_]] = []
                all_docs[global_id]["File"] = filename
            if line.startswith("KW  - "):
                keywords_started = True
                continue
            if keywords_started and re.search("^[A-Z0-9]{2}  - ", line):
                keywords_started = False
            if keywords_started:
                all_docs[global_id]["keywords"].append(line.strip().lower())
            for start_symbols in mappings_for_columns:
                if start_symbols == "KW  - ":
                    continue
                column_name = mappings_for_columns[start_symbols]
                if line.startswith(start_symbols):
                    all_docs[global_id][column_name].append(line.replace(start_symbols, "").strip())

In [22]:
for _id in all_docs:
    for column in all_docs[_id]:
        if type(all_docs[_id][column]) != str:
            all_docs[_id][column] = ";".join(all_docs[_id][column])

In [23]:
df = pd.DataFrame(all_docs.values())

In [24]:
for i in range(len(df)):
    if df["year"].values[i].strip():
        df["year"].values[i] = int(df["year"].values[i])

In [25]:
excel_writer.ExcelWriter().save_df_in_excel(df, "../tmp/combined_egm_data.xlsx")

Saving...
Saved to ../tmp/combined_egm_data.xlsx


# Check Lab studies and Geo names

In [2]:
df = excel_reader.ExcelReader().read_df_from_excel("../tmp/combined_egm_data.xlsx")

Read file ../tmp/combined_egm_data.xlsx: 0.11s
Processed file ../tmp/combined_egm_data.xlsx: 0.06s


In [3]:
df["identificators"] = ""

In [5]:
# if needed
df["keywords"] = ""
df["title"] = df["Title"]
df["abstract"] = df["Abstract"]

In [4]:
df["File"].value_counts()

Included (abstract)      100
Lab studies              100
Non-causal studies       100
High-income countries    100
No intervention          100
Name: File, dtype: int64

In [4]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler
search_engine_inverted_index = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
search_engine_inverted_index.create_inverted_index(df)

Processed 0 articles
Processed 499 articles
Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviations
Processed 21000 abbreviations
Processed 24000 abbreviations
Processed 27000 abbreviations
Processed 27129 abbreviations


In [5]:
_all_column_filler = all_column_filler.AllColumnFiller()
df = _all_column_filler.fill_columns_for_df(
        df, search_engine_inverted_index, _abbreviations_resolver, settings_json = {"columns":[
            #{"column_filler_class":"GeoNameFinder", "world_bank_regions_file": "../data/IncomeLevelDivisionCountries.xlsx"},
            {"column_filler_class": "StudyTypeLabeller", "folder": "../model/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1",
             "meta_folder": "../model/study_type_multi_meta_new_data_with_keywords_agg_more_sub_groups_more_data_1",
             "scibert_model_folder": "../model/scibert_scivocab_uncased", "model_with_agg": True},
        ]})

Started processing  {'column_filler_class': 'StudyTypeLabeller', 'folder': '../model/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1', 'meta_folder': '../model/study_type_multi_meta_new_data_with_keywords_agg_more_sub_groups_more_data_1', 'scibert_model_folder': '../model/scibert_scivocab_uncased', 'model_with_agg': True}
High level label: Book chapter
High level label: Field study
High level label: Greenhouse study
High level label: Impact evaluation
High level label: Laboratory study
High level label: Meta analysis
High level label: Modelling study
High level label: Observational study
High level label: Randomized controlled trials
High level label: Review paper
High level label: Systematic review
High level label: Interview
High level label: Survey/Questionnaire
High level label: Experimental research
High level label: Economic simulation
High level label: Landscape/environmental simulation
High level label: Policy analysis
Labelled articles with outcomes: 21

INFO:tensorflow:Using config: {'_model_dir': '../model/study_type_multi_new_data_with_keywords_agg_more_sub_groups_more_data_1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023F28DA26A0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Config is done
INFO:tensorflow:Writing example 0 of 500


INFO:tensorflow:Writing example 0 of 500


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 

In [7]:
excel_writer.ExcelWriter().save_df_in_excel(df, "../tmp/combined_data_processed_new.xlsx")

Saving...
Saved to ../tmp/combined_data_processed_new.xlsx


# Lab studies evaluation

In [15]:
lab_study_true_vals = []
lab_study_true_pred = []
for i in range(len(df)):
    if df["File"].values[i] == "Lab studies":
        lab_study_true_vals.append(1)
    else:
        lab_study_true_vals.append(0)
    if "Laboratory study" in df["study_type"].values[i]:
        lab_study_true_pred.append(1)
    else:
        lab_study_true_pred.append(0)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(lab_study_true_vals, lab_study_true_pred))  
print(classification_report(lab_study_true_vals, lab_study_true_pred))

[[389  11]
 [ 74  26]]
              precision    recall  f1-score   support

           0       0.84      0.97      0.90       400
           1       0.70      0.26      0.38       100

   micro avg       0.83      0.83      0.83       500
   macro avg       0.77      0.62      0.64       500
weighted avg       0.81      0.83      0.80       500



# High income countries evaluation

In [9]:
geo_locations_info_true_vals = []
geo_locations_info_pred = []
for i in range(len(df)):
    if df["File"].values[i] == 'Lab studies':
        continue
    if df["File"].values[i] == 'High-income countries':
        geo_locations_info_true_vals.append(1)
    else:
        geo_locations_info_true_vals.append(0)
    if 'Low-income countries(995$ or less)' in df["world_bankdivision_regions"].values[i] or 'Middle-income countries($996 to $3,895)' in df["world_bankdivision_regions"].values[i] or "Upper-middle-income countries ($3,896 to $12,055)" in df["world_bankdivision_regions"].values[i]:
        geo_locations_info_pred.append(0)
    elif len(df["world_bankdivision_regions"].values[i]) == 0:
        geo_locations_info_pred.append(0)
    else:
        geo_locations_info_pred.append(1)

In [10]:
geo_locations_info_true_vals = []
geo_locations_info_pred = []
for i in range(len(df)):
    if df["File"].values[i] == 'Lab studies':
        continue
    if df["File"].values[i] == 'High-income countries':
        geo_locations_info_true_vals.append(1)
    else:
        if "LMICs" not in df["world_bankdivision_regions"].values[i] and "Transitional countries" in df["world_bankdivision_regions"].values[i]:
            geo_locations_info_true_vals.append(1)
        else:
            geo_locations_info_true_vals.append(0)
        #geo_locations_info_true_vals.append(0)
    if "LMICs" in df["world_bankdivision_regions"].values[i]:
        geo_locations_info_pred.append(0)
    elif len(df["world_bankdivision_regions"].values[i]) == 0:
        geo_locations_info_pred.append(0)
    else:
        geo_locations_info_pred.append(1)

In [9]:
#baseline
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(geo_locations_info_true_vals, geo_locations_info_pred))
print(classification_report(geo_locations_info_true_vals, geo_locations_info_pred))

[[279  21]
 [ 23  77]]
              precision    recall  f1-score   support

           0       0.92      0.93      0.93       300
           1       0.79      0.77      0.78       100

   micro avg       0.89      0.89      0.89       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.89      0.89      0.89       400



In [11]:
# recalculated with labels changed for transitional countries not excluded in the dataset
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(geo_locations_info_true_vals, geo_locations_info_pred))
print(classification_report(geo_locations_info_true_vals, geo_locations_info_pred))

[[279   6]
 [ 23  92]]
              precision    recall  f1-score   support

           0       0.92      0.98      0.95       285
           1       0.94      0.80      0.86       115

   micro avg       0.93      0.93      0.93       400
   macro avg       0.93      0.89      0.91       400
weighted avg       0.93      0.93      0.93       400



In [11]:
geo_locations_info_true_vals = []
geo_locations_info_pred = []
for i in range(len(df)):
    if df["File"].values[i] != 'Included (abstract)':
        continue
    if df["File"].values[i] == 'High-income countries':
        geo_locations_info_true_vals.append(1)
    else:
        geo_locations_info_true_vals.append(0)
    if 'Low-income countries(995$ or less)' in df["world_bankdivision_regions"].values[i] or 'Middle-income countries($996 to $3,895)' in df["world_bankdivision_regions"].values[i] or "Upper-middle-income countries ($3,896 to $12,055)" in df["world_bankdivision_regions"].values[i]:
        geo_locations_info_pred.append(0)
    elif len(df["world_bankdivision_regions"].values[i]) == 0:
        geo_locations_info_pred.append(0)
    else:
        geo_locations_info_pred.append(1)

In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(geo_locations_info_true_vals, geo_locations_info_pred)

0.97

# Geo names extraction accuracy

In [14]:
geo_names_df = excel_reader.ExcelReader().read_df_from_excel("../tmp/labelled_geo_names.xlsx")

Read file ../tmp/labelled_geo_names.xlsx: 0.07s
Processed file ../tmp/labelled_geo_names.xlsx: 0.08s


In [15]:
geo_names_df.columns

Index(['abstract', 'comments', 'countries_mentioned', 'country_groups',
       'districts', 'edit_url', 'geo_regions', 'keywords', 'labelled',
       'provinces', 'responsible_person', 'tags', 'title'],
      dtype='object')

In [16]:
print("Labelled data: %d All data: %d" % (len(geo_names_df[geo_names_df["labelled"] == 1]), len(geo_names_df)))

Labelled data: 169 All data: 500


In [28]:
geo_names_df_with_labelled = geo_names_df[geo_names_df["labelled"] == 1]

In [25]:
geo_names_df_wo_labelled = geo_names_df_with_labelled[["title","abstract","keywords"]].copy()
geo_names_df_wo_labelled["identificators"] = ""

In [26]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler
search_engine_inverted_index_geo_names = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True)
search_engine_inverted_index_geo_names.create_inverted_index(geo_names_df_wo_labelled)

Processed 0 articles
Processed 168 articles
Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviations
Processed 21000 abbreviations
Processed 24000 abbreviations
Processed 27000 abbreviations
Processed 27129 abbreviations


In [27]:
_all_column_filler = all_column_filler.AllColumnFiller()
geo_names_df_wo_labelled = _all_column_filler.fill_columns_for_df(
        geo_names_df_wo_labelled, search_engine_inverted_index_geo_names, _abbreviations_resolver, settings_json = {"columns":[
            {"column_filler_class":"GeoNameFinder", "world_bank_regions_file": "../data/IncomeLevelDivisionCountries.xlsx"},
        ]})

Started processing  {'column_filler_class': 'GeoNameFinder', 'world_bank_regions_file': '../data/IncomeLevelDivisionCountries.xlsx'}
Read file ../data/GeoRegions.xlsx: 0.02s
Processed file ../data/GeoRegions.xlsx: 0.00s
Read file ../data/map_additional_regions.xlsx: 0.01s
Processed file ../data/map_additional_regions.xlsx: 0.00s
Read file ../data/IncomeLevelDivisionCountries.xlsx: 0.01s
Processed file ../data/IncomeLevelDivisionCountries.xlsx: 0.00s
Read file ../data/map_additional_country_names.xlsx: 0.00s
Processed file ../data/map_additional_country_names.xlsx: 0.00s
Read file ../data/map_country_adjectives.xlsx: 0.01s
Processed file ../data/map_country_adjectives.xlsx: 0.01s
Read file ../data/map_currencies.xlsx: 0.01s
Processed file ../data/map_currencies.xlsx: 0.00s
Read file ../data/map_country_groups.xlsx: 0.01s
Processed file ../data/map_country_groups.xlsx: 0.01s
Read file ../data/map_country_groups_income_level.xlsx: 0.00s
Processed file ../data/map_country_groups_income_lev

In [30]:
def get_mean_intersection_over_union(predicted_vals, true_vals):
    cnt = 0
    cnt_intersection_over_union = 0.0
    for pred, true in zip(predicted_vals, true_vals):
        cnt += 1
        cnt_intersect = len(set(pred).intersection(set(true)))
        cnt_union = len(set(pred).union(set(true)))
        if cnt_union == 0:
            cnt_intersection_over_union += 1
        else:
            cnt_intersection_over_union += (cnt_intersect/cnt_union)
    return cnt_intersection_over_union/cnt if cnt else 0.0

for column in ['countries_mentioned', 'country_groups','districts', 'geo_regions', 'provinces']:
    print("Column: ", column)
    print(get_mean_intersection_over_union(geo_names_df_wo_labelled[column].values, geo_names_df_with_labelled[column].values))

Column:  countries_mentioned
0.9119822485207102
Column:  country_groups
0.945759368836292
Column:  districts
0.9644970414201184
Column:  geo_regions
0.9452662721893491
Column:  provinces
0.935897435897436


In [31]:
geo_names_df_wo_labelled_restricted_by_title = geo_names_df_wo_labelled.copy()

In [32]:
from text_processing import search_engine_insensitive_to_spelling
from text_processing import all_column_filler
search_engine_inverted_index_geo_names_title = search_engine_insensitive_to_spelling.SearchEngineInsensitiveToSpelling(
        load_abbreviations = True, columns_to_process=["title"])
search_engine_inverted_index_geo_names_title.create_inverted_index(geo_names_df_wo_labelled_restricted_by_title)

Processed 0 articles
Processed 168 articles
Processed 0 abbreviations
Processed 3000 abbreviations
Processed 6000 abbreviations
Processed 9000 abbreviations
Processed 12000 abbreviations
Processed 15000 abbreviations
Processed 18000 abbreviations
Processed 21000 abbreviations
Processed 24000 abbreviations
Processed 27000 abbreviations
Processed 27129 abbreviations


In [34]:
_all_column_filler = all_column_filler.AllColumnFiller()
geo_names_df_wo_labelled_restricted_by_title = _all_column_filler.fill_columns_for_df(
        geo_names_df_wo_labelled_restricted_by_title, search_engine_inverted_index_geo_names_title, _abbreviations_resolver, settings_json = {"columns":[
            {"column_filler_class":"GeoNameFinder", "world_bank_regions_file": "../data/IncomeLevelDivisionCountries.xlsx",
            "columns_to_process":["title"], "prefix_for_columns": "title_", "check_only_country_names_and_divisions": True},
        ]})

Started processing  {'column_filler_class': 'GeoNameFinder', 'world_bank_regions_file': '../data/IncomeLevelDivisionCountries.xlsx', 'columns_to_process': ['title'], 'prefix_for_columns': 'title_', 'check_only_country_names_and_divisions': True}
Read file ../data/GeoRegions.xlsx: 0.01s
Processed file ../data/GeoRegions.xlsx: 0.01s
Read file ../data/map_additional_regions.xlsx: 0.01s
Processed file ../data/map_additional_regions.xlsx: 0.00s
Read file ../data/IncomeLevelDivisionCountries.xlsx: 0.01s
Processed file ../data/IncomeLevelDivisionCountries.xlsx: 0.01s
Read file ../data/map_additional_country_names.xlsx: 0.01s
Processed file ../data/map_additional_country_names.xlsx: 0.00s
Read file ../data/map_country_adjectives.xlsx: 0.01s
Processed file ../data/map_country_adjectives.xlsx: 0.01s
Read file ../data/map_currencies.xlsx: 0.01s
Processed file ../data/map_currencies.xlsx: 0.00s
Read file ../data/map_country_groups.xlsx: 0.01s
Processed file ../data/map_country_groups.xlsx: 0.01s
R

In [35]:
geo_names_df_wo_labelled_restricted_by_title.head()

,title,abstract,keywords,identificators,countries_mentioned,country_codes,provinces,districts,country_groups,geo_regions,world_bankdivision_regions,title_countries_mentioned,title_country_codes,title_provinces,title_districts,title_country_groups,title_geo_regions,title_world_bankdivision_regions
331,The effect of education based on reasoned acti...,Introduction: Studies showed that about 25 per...,analytical techniques;behavior;studies;man;hom...,,[Iran],[IR],[Iran/Fars],[Iran/Fars/Sepidan],[developing countries],"[Middle East and North Africa, Asia]",[LMICs],[Iran],[IR],[Iran/Fars],[Iran/Fars/Sepidan],[],[Middle East and North Africa],[LMICs]
332,Motivational interview on having Pap test amon...,"Background. Cognitive and mental factors, such...",counseling;fisher exact test;health attitudes;...,,[Iran],[IR],[Iran/Markazi],[Iran/Markazi/Shazand],[],[Middle East and North Africa],[LMICs],[],[],[],[],[],[],[]
333,Effectiveness of intervention due to feedback ...,Background: The limited supply of red blood ce...,red blood cell transfusion;blood transportatio...,,[Iran],[IR],[Iran/Razavi Khorasan],[Iran/Razavi Khorasan/Mashhad],[],[Middle East and North Africa],[LMICs],[],[],[],[],[],[],[]
334,Evaluation of the clinical pharmacist role in ...,Background Lower urinary tract symptoms due to...,"aged;aged, 80 and over;humans;jordan;*lower ur...",,[Jordan],[JO],[Jordan/Amman],[Jordan/Amman/Amman],[],[Middle East and North Africa],[LMICs],[],[],[],[],[],[],[]
335,Use of propensity score matching to create cou...,The design of HIV prevention trials in the con...,hiv prevention;vaccine effectiveness;hepatitis...,,[Uganda],[UG],[],[],[],[Sub-Saharan Africa],[LMICs],[],[],[],[],[],[],[]


In [37]:
for i in range(len(geo_names_df_wo_labelled_restricted_by_title)):
    if geo_names_df_wo_labelled_restricted_by_title["title_countries_mentioned"].values[i]:
        geo_names_df_wo_labelled_restricted_by_title["countries_mentioned"].values[i] = geo_names_df_wo_labelled_restricted_by_title["title_countries_mentioned"].values[i]
        for column in ["provinces", "districts"]:
            new_array = []
            for district in geo_names_df_wo_labelled_restricted_by_title[column].values[i]:
                if district.split("/")[0] in geo_names_df_wo_labelled_restricted_by_title["countries_mentioned"].values[i]:
                    new_array.append(district)
            geo_names_df_wo_labelled_restricted_by_title[column].values[i] = new_array

In [38]:
for column in ['countries_mentioned', 'country_groups','districts', 'geo_regions', 'provinces']:
    print("Column: ", column)
    print(get_mean_intersection_over_union(geo_names_df_wo_labelled_restricted_by_title[column].values, geo_names_df_with_labelled[column].values))

Column:  countries_mentioned
0.8982448725363706
Column:  country_groups
0.945759368836292
Column:  districts
0.9704142011834319
Column:  geo_regions
0.9452662721893491
Column:  provinces
0.9437869822485208
